In [1]:
from sqlalchemy import create_engine

In [2]:
import pandas as pd

In [3]:
user, password, host, port, db = "root", "root", "localhost", "5432", "ny_taxi"
conn = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [12]:
query = '''
SELECT COUNT(1) FROM taxi_trips as t
WHERE DATE(t.lpep_pickup_datetime) = '2019-09-18';
'''

pd.read_sql(query, con=conn)


,count
0,15767


In [14]:
query = '''
SELECT DATE(t.lpep_pickup_datetime) FROM taxi_trips as t
WHERE t.trip_distance = (SELECT MAX(trip_distance) FROM taxi_trips);
'''

pd.read_sql(query, con=conn)


,date
0,2019-09-26


In [4]:
query = '''
SELECT * FROM taxi_zones as t
LIMIT 100;
'''

pd.read_sql(query, con=conn)


,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
95,96,Queens,Forest Park/Highland Park,Boro Zone
96,97,Brooklyn,Fort Greene,Boro Zone
97,98,Queens,Fresh Meadows,Boro Zone
98,99,Staten Island,Freshkills Park,Boro Zone


In [5]:
query = '''
SELECT * FROM taxi_trips as t
LIMIT 100;
'''

pd.read_sql(query, con=conn)

,index,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,0,2,2019-09-01 00:10:53,2019-09-01 00:23:46,N,1,65,189,5,2.00,...,0.5,0.5,2.36,0.00,None,0.3,14.16,1,1,0.0
1,1,2,2019-09-01 00:31:22,2019-09-01 00:44:37,N,1,97,225,5,3.20,...,0.5,0.5,0.00,0.00,None,0.3,13.30,2,1,0.0
2,2,2,2019-09-01 00:50:24,2019-09-01 01:03:20,N,1,37,61,5,2.99,...,0.5,0.5,0.00,0.00,None,0.3,13.30,2,1,0.0
3,3,2,2019-09-01 00:27:06,2019-09-01 00:33:22,N,1,145,112,1,1.73,...,0.5,0.5,1.50,0.00,None,0.3,10.30,1,1,0.0
4,4,2,2019-09-01 00:43:23,2019-09-01 00:59:54,N,1,112,198,1,3.42,...,0.5,0.5,3.06,0.00,None,0.3,18.36,1,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,2,2019-09-01 00:40:55,2019-09-01 00:48:35,N,1,95,160,1,2.24,...,0.5,0.5,0.00,0.00,None,0.3,10.30,2,1,0.0
96,96,2,2019-09-01 00:13:52,2019-09-01 00:21:47,N,1,75,151,1,2.04,...,0.5,0.5,1.96,0.00,None,0.3,11.76,1,1,0.0
97,97,2,2019-09-01 00:37:24,2019-09-01 01:02:31,N,1,41,182,1,7.77,...,0.5,0.5,5.46,0.00,None,0.3,32.76,1,1,0.0
98,98,2,2019-09-01 00:22:15,2019-09-01 00:29:52,N,1,74,260,1,4.99,...,0.5,0.5,2.58,6.12,None,0.3,25.00,1,1,0.0


In [18]:
query = '''
SELECT t4."Borough", SUM(t4.total_amount) as total_sum
FROM
(
    SELECT t3.*
    FROM
    (
        SELECT * 
        FROM taxi_trips as t1 
        JOIN taxi_zones as t2
        ON t1."PULocationID" = t2."LocationID"
    ) as t3
    WHERE DATE(t3.lpep_pickup_datetime) = '2019-09-18' AND t3."Borough" <> 'Unknown'
) as t4
GROUP BY t4."Borough"
HAVING SUM(t4.total_amount) >= 50000;
'''

pd.read_sql(query, con=conn)


,Borough,total_sum
0,Brooklyn,96333.24
1,Manhattan,92271.30
2,Queens,78671.71


In [5]:
query = '''
SELECT *
FROM
(
    SELECT *
    FROM
    taxi_trips as t1
    JOIN taxi_zones as t2
    ON t1."PULocationID" = t2."LocationID"
) as t3
WHERE t3.lpep_pickup_datetime >= '2019-09-01' AND t3.lpep_pickup_datetime < '2019-10-01' AND t3."Zone" = 'Astoria';
'''

pd.read_sql(query, con=conn)


,index,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,...,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,LocationID,Borough,Zone,service_zone
0,5,2.0,2019-09-01 00:55:37,2019-09-01 01:17:02,N,1.0,7,41,1.0,6.26,...,None,0.3,28.42,2.0,1.0,0.0,7,Queens,Astoria,Boro Zone
1,15,1.0,2019-09-01 00:35:43,2019-09-01 00:47:21,N,1.0,7,83,1.0,2.10,...,None,0.3,13.00,1.0,1.0,0.0,7,Queens,Astoria,Boro Zone
2,20,2.0,2019-09-01 00:58:19,2019-09-01 01:07:00,N,1.0,7,179,1.0,1.43,...,None,0.3,9.30,2.0,1.0,0.0,7,Queens,Astoria,Boro Zone
3,44,2.0,2019-09-01 00:15:10,2019-09-01 00:18:22,N,1.0,7,7,1.0,0.52,...,None,0.3,5.30,2.0,1.0,0.0,7,Queens,Astoria,Boro Zone
4,45,2.0,2019-09-01 00:21:37,2019-09-01 00:33:31,N,1.0,7,226,1.0,1.85,...,None,0.3,11.30,2.0,1.0,0.0,7,Queens,Astoria,Boro Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18257,448977,NaN,2019-09-30 20:17:00,2019-09-30 20:56:00,None,NaN,7,22,NaN,16.67,...,None,0.0,68.10,NaN,NaN,NaN,7,Queens,Astoria,Boro Zone
18258,448477,NaN,2019-09-30 14:26:00,2019-09-30 15:03:00,None,NaN,7,28,NaN,9.86,...,None,0.0,38.97,NaN,NaN,NaN,7,Queens,Astoria,Boro Zone
18259,448546,NaN,2019-09-30 15:00:00,2019-09-30 15:29:00,None,NaN,7,47,NaN,8.98,...,None,0.0,40.44,NaN,NaN,NaN,7,Queens,Astoria,Boro Zone
18260,448578,NaN,2019-09-30 15:22:00,2019-09-30 15:51:00,None,NaN,7,247,NaN,8.90,...,None,0.0,33.06,NaN,NaN,NaN,7,Queens,Astoria,Boro Zone
